# Select Your IPTS

In [1]:
from __code import system
system.System.select_working_dir()
from __code.__all import custom_style
custom_style.style()

** Using Debugging Mode! **


HBox(children=(Label(value='Select Working Folder', layout=Layout(width='20%')), Select(index=8, options=('BraggEdge', 'Brianne', 'DOE_letters', 'FITSimages', 'IPTS-12534', 'IPTS_13639_resonance_shawn', 'IPTS_14832', 'IPTS_16212', 'IPTS_16259', 'IPTS_16475', 'IPTS_16893', 'IPTS_17099', 'IPTS_17685', 'IPTS_17981', 'IPTS_18806', 'IPTS_19037', 'IPTS_19558', 'IPTS_19621', 'IPTS_19626', 'IPTS_test_normalization', 'IPTS_testing_time_spectra', 'ImagesToRegisterForGian', 'Indu', 'Keita_code', 'Pour_ibeatles_video', 'ResonanceImaging', 'SENSITIVE', 'SarmaGUI', 'SphereCoordinates3D', 'TempData', 'gian', 'iBeatles_debugging'), value='IPTS_16259')))

# Description

In [19]:
import warnings
warnings.filterwarnings('ignore')

1. Load images
2. Display images and histogram 
3. Select what you want to fix
    * pixels below a given threshold
    * pixels above a given threshold
4. Replace by value of choice
    * np.NaN
    * given value
5. export new images 

In [2]:
from __code.fix_images import FixImages

In [3]:
%matplotlib notebook
%matplotlib inline

# Select Images

In [4]:
_o_fix = FixImages(working_dir=system.System.get_working_dir())
_o_fix.select_images()

HTML(value='Please wait...')

VBox(children=(Label(value='Select Images ...', layout=Layout(width='250px')), HBox(children=(HBox(children=(Text(value='/Volumes/my_book_thunderbolt_duo/IPTS/gian', description='Location: ', layout=Layout(width='300px'), placeholder=''), Button(description='Jump', layout=Layout(margin='5px 10px', width='100px'), style=ButtonStyle())), layout=Layout(border='1px solid lightgrey', margin='5px 50px 5px 5px', padding='3px')),)), VBox(children=(SelectMultiple(description='Select', layout=Layout(display='flex', flex_flow='column', width='750px'), options=(' .', ' ..', ' Total_barn_90_0557.fits          |     Thu Dec 14 13:11:31 2017', ' Total_barn_90_0973.fits          |     Thu Dec 14 13:11:27 2017', ' Total_barn_90_1170.fits          |     Thu Dec 14 13:11:29 2017', ' Total_barns_0degree0557.fits     |     Thu Dec 14 13:11:17 2017', ' Total_barns_0degree0973.fits     |     Thu Dec 14 13:11:11 2017', ' Total_barns_0degree1170.fits     |     Thu Dec 14 13:11:15 2017'), value=()), HBox(children=(Button(description='Enter directory', layout=Layout(margin='5px 40px'), style=ButtonStyle()), Button(description='Select', layout=Layout(margin='5px 40px'), style=ButtonStyle())))), layout=Layout(border='1px solid lightgrey', margin='5px', padding='10px'))))

# Give statistics 

In [9]:
import os
import numpy as np
from ipywidgets import widgets

from collections import namedtuple

In [28]:
Statistics = namedtuple('Statistics',
                        'total_pixels nbr_negative percentage_negative nbr_infinite percentage_infinite nbr_nan percentage_nan')


In [24]:
_o_fix.load()

IntProgress(value=0, max=3)

In [29]:
 def get_statistics_of_roi_cleaned(data):

        _data = data
        _result = np.where(_data < 0)

        nbr_negative = len(_result[0])
        total = np.size(data)
        percentage_negative = (nbr_negative / total) * 100

        _result_inf = np.where(np.isinf(_data))
        nbr_inf = len(_result_inf[0])
        percentage_inf = (nbr_inf / total) * 100
        
        nan_values = np.where(np.isnan(_data))
        nbr_nan = len(nan_values[0])
        percentage_nan = (nbr_nan / total) * 100
        
        stat = Statistics(total_pixels=total,
                          nbr_negative=nbr_negative,
                         percentage_negative=percentage_negative,
                         nbr_infinite=nbr_inf,
                         percentage_infinite=percentage_inf,
                         nbr_nan=nbr_nan,
                         percentage_nan=percentage_nan)
        return stat


In [34]:
import matplotlib.pyplot as plt

In [84]:
list_files = _o_fix.list_files
data = _o_fix.data

def give_statistics(index):
    
    _file = os.path.basename(list_files[index])
    _data = data[index]

    def on_change(change):
        print(change)
        print("here")
    
    stat = get_statistics_of_roi_cleaned(_data)
    
    number_of_pixels = stat.total_pixels
    negative_values = stat.nbr_negative
    negative_percentage = stat.percentage_negative
    infinite_values = stat.nbr_infinite
    infinite_percentage = stat.percentage_infinite
    nan_values = stat.nbr_nan
    nan_percentage = stat.percentage_nan
    
    box1 = widgets.HBox([widgets.Label("File Name:"),
                       widgets.Label(_file,
                                    layout=widgets.Layout(width='80%'))])
    box2 = widgets.HBox([widgets.Label("Total number of pixels:",
                                      layout=widgets.Layout(width='15%')),
                        widgets.Label(str(number_of_pixels))])
    
    box3 = widgets.HBox([widgets.Label("Negative values:",
                                      layout=widgets.Layout(width='10%')),
                        widgets.Label("{} pixels ({:.3}%)".format(negative_values, negative_percentage),
                                     layout=widgets.Layout(width='15%')),
                        widgets.Label("to replace by"),
                        widgets.Dropdown(options=['NaN', '0'],
                                        value='NaN')])
    neg_widgets = box3.children[3]
    neg_widgets.observe(on_change)
    
    box4 = widgets.HBox([widgets.Label("Infinite values:",
                                      layout=widgets.Layout(width='10%')),
                        widgets.Label("{} pixels ({:.3}%)".format(infinite_values, infinite_percentage),
                                     layout=widgets.Layout(width='15%')),
                        widgets.Label("to replace by"),
                        widgets.Dropdown(options=['NaN', '0'],
                                        value='NaN')])

    box5 = widgets.HBox([widgets.Label("NaN values:",
                                      layout=widgets.Layout(width='10%')),
                        widgets.Label("{} pixels ({:.3}%)".format(nan_values, nan_percentage),
                                     layout=widgets.Layout(width='15%')),
                                                widgets.Label("to replace by"),
                        widgets.Dropdown(options=['NaN', '0'],
                                        value='NaN')])

    
    vertical_box = widgets.VBox([box1, box2, box3, box4, box5])
    display(vertical_box)
    
    _result_inf = np.where(np.isinf(_data))
    _data[_result_inf] = np.NaN
    
    fig, [[ax0, ax1], [ax2, ax3]] = plt.subplots(ncols=2, nrows=2,
                                   figsize=(15, 10),
                                   num=os.path.basename(_file))

    # plt.title(os.path.basename(_files[index]))
    cax0 = ax0.imshow(_data, cmap='viridis', interpolation=None)
    ax0.set_title("Raw Image")
    tmp1 = fig.colorbar(cax0, ax=ax0)  # colorbar

    ax1.hist(_data.ravel(), range=(np.nanmin(_data), np.nanmax(_data)), bins=256)
    ax1.set_title("Raw Histogram")

    cax2 = ax2.imshow(_data, cmap='viridis', interpolation=None)
    ax2.set_title("New Image")
    tmp1 = fig.colorbar(cax2, ax=ax2)  # colorbar

    ax3.hist(_data.ravel(), range=(np.nanmin(_data), np.nanmax(_data)), bins=256)
    ax3.set_title("New Histogram")

    fig.tight_layout()
    
    
tmp3 = widgets.interact(give_statistics,
                        index=widgets.IntSlider(min=0,
                                                max=len(list_files) - 1,
                                                step=1,
                                                value=0,
                                                description='File Index',
                                                continuous_update=False),
                        )

interactive(children=(IntSlider(value=0, continuous_update=False, description='File Index', max=2), Output()), _dom_classes=('widget-interact',))

# Display Images 

In [ ]:
_o_fix.load()

In [ ]:
_o_fix.display_histo_and_images()

In [ ]:
from ipywidgets import widgets

In [ ]:
self = _o_fix

_data = self.data
_files = self.list_files
_full_statistics = self.full_statistics

[height, width] = np.shape(self.data[0])

def _plot_images(index):
    _file_name = _files[index]
    fig, (ax0, ax1) = plt.subplots(ncols=2,
                                   figsize=(10, 5),
                                   num=os.path.basename(_file_name))

    # plt.title(os.path.basename(_files[index]))
    cax0 = ax0.imshow(_data[index], cmap='viridis', interpolation=None)
    ax0.set_title("Raw Image")
    tmp1 = fig.colorbar(cax0, ax=ax0)  # colorbar

    _min = np.nanmin(_data[index])
    _max = np.nanmax(_data[index])

    print(_min)
    print(_max)
    
    ax1.hist(_data[index].ravel(), range=(_min, _max), bins=256)

    fig.tight_layout()


tmp3 = widgets.interact(_plot_images,
                        index=widgets.IntSlider(min=0,
                                                max=len(self.list_files) - 1,
                                                step=1,
                                                value=0,
                                                description='File Index',
                                                continuous_update=False),
                        )


# Turn off all negative values 

In [ ]:
_o_fix.load()
_o_fix.remove_negative_values()
_o_fix.display_images()

# Display Images for Publication 

In [ ]:
%matplotlib notebook

In [ ]:
_o_fix.display_images_pretty()

# Export

In [ ]:
_o_fix.select_output_folder()

In [ ]:
_o_fix.export()